# 📖 Chapter 03 — Gemini API Testing

## 🎯 Objectives

In this chapter, we will test the Google Gemini API to ensure it works correctly before integrating it into our RAG pipeline.

We will test both the text generation (Gemini Pro) and embedding generation (Gemini Embedding) capabilities.

## 🔑 Step 01 — API Key Setup
Set up the Gemini API key from Google AI Studio and configure environment variables.

In [1]:
import os
import time
import google.generativeai as genai

from dotenv import load_dotenv
from src.utils.emoji_log import success, error, info, task, data, warn
load_dotenv()

True

In [2]:
api_key = os.getenv("GOOGLE_API_KEY")

In [3]:
if api_key:
    genai.configure(api_key=api_key)
    success("API Key loaded successfully!")
    info(f"API_KEY: {api_key[:8]}...{api_key[-4:]}")
else:
    error("API Key not found! Please check your .env file.")

✅ API Key loaded successfully!
💬 API_KEY: AIzaSyCU...9UL0


## 🤖 Step 02 — Test Gemini Pro (Text Generation)
Test the Gemini Pro model for text generation with a simple query.

In [4]:
# List all useful models
for model in genai.list_models():
    if "generateContent" in model.supported_generation_methods:
        print(f"Model: {model.name}")
        print(f"Display Name: {model.display_name}")
        print(f"Description: {model.description}")

Model: models/gemini-2.5-flash
Display Name: Gemini 2.5 Flash
Description: Stable version of Gemini 2.5 Flash, our mid-size multimodal model that supports up to 1 million tokens, released in June of 2025.
Model: models/gemini-2.5-pro
Display Name: Gemini 2.5 Pro
Description: Stable release (June 17th, 2025) of Gemini 2.5 Pro
Model: models/gemini-2.0-flash-exp
Display Name: Gemini 2.0 Flash Experimental
Description: Gemini 2.0 Flash Experimental
Model: models/gemini-2.0-flash
Display Name: Gemini 2.0 Flash
Description: Gemini 2.0 Flash
Model: models/gemini-2.0-flash-001
Display Name: Gemini 2.0 Flash 001
Description: Stable version of Gemini 2.0 Flash, our fast and versatile multimodal model for scaling across diverse tasks, released in January of 2025.
Model: models/gemini-2.0-flash-exp-image-generation
Display Name: Gemini 2.0 Flash (Image Generation) Experimental
Description: Gemini 2.0 Flash (Image Generation) Experimental
Model: models/gemini-2.0-flash-lite-001
Display Name: Gemini

In [5]:
# Establish Gemini model
model = genai.GenerativeModel(model_name="gemini-flash-latest")

In [6]:
# Test question
test_question = "Please introduce Taiwan attraction in a sentence."

task(f"Testing Gemini Pro with question: '{test_question}'")

🚀 Testing Gemini Pro with question: 'Please introduce Taiwan attraction in a sentence.'


In [7]:
response = model.generate_content(test_question)

success("Response received!")
info("Generated text:")
print(response.text)
data(f"Response length: {len(response.text)} characters")

✅ Response received!
💬 Generated text:
Taiwan is a captivating destination renowned for its delicious street food, majestic mountain landscapes, and famously welcoming people.
📊 Response length: 136 characters


## 🧮 Step 03 — Test Gemini Embedding
Test the Gemini Embedding API to generate vector embeddings for text.

In [8]:
# List embedding model
for model in genai.list_models():
    if 'embedContent' in model.supported_generation_methods:
        print(f"Model: {model.name}")

Model: models/embedding-001
Model: models/text-embedding-004
Model: models/gemini-embedding-exp-03-07
Model: models/gemini-embedding-exp
Model: models/gemini-embedding-001


In [9]:
for i in range(25):
    try:
        result = genai.embed_content(
            model="models/text-embedding-004", content=f"Test{i}"
        )
        success(f"Call {i+1} successful")
        time.sleep(1)
    except Exception as e:
        error(f"Call {i+1} failed: {e}")
        break

✅ Call 1 successful
✅ Call 2 successful
✅ Call 3 successful
✅ Call 4 successful
✅ Call 5 successful
✅ Call 6 successful
✅ Call 7 successful
✅ Call 8 successful
✅ Call 9 successful
✅ Call 10 successful
✅ Call 11 successful
✅ Call 12 successful
✅ Call 13 successful
✅ Call 14 successful
✅ Call 15 successful
✅ Call 16 successful
✅ Call 17 successful
✅ Call 18 successful
✅ Call 19 successful
✅ Call 20 successful
✅ Call 21 successful
✅ Call 22 successful
✅ Call 23 successful
✅ Call 24 successful
✅ Call 25 successful


## 📊 Step 04 — Verify Embedding Dimensions
Check the embedding vector dimensions and verify they are consistent.

In [10]:
test_texts = [
    "Taipei 101 is the tallest building in Taiwan",
    "台北101是台灣最高的建築",
    "Short",
    "A very long text with many words to test if the embedding dimension stays consistent across different text lengths and languages including English and Chinese characters and special symbols like !@#$%",
]

task("Testing embedding dimensions across different texts...")

dimensions = []

for i, text in enumerate(test_texts, start=1):
    result = genai.embed_content(model="models/text-embedding-004", content=text)
    # result is a dict and the embedding key saves all coordinates
    embedding = result["embedding"]
    dimensions.append(len(embedding))

    data(f"Text {i} (length: {len(text)} chars) → Embedding dim: {len(embedding)}")
    info(f"First 5 values: {embedding[:5]}")

🚀 Testing embedding dimensions across different texts...


📊 Text 1 (length: 44 chars) → Embedding dim: 768
💬 First 5 values: [0.01981311, 0.023207255, -0.009134806, 0.018078767, 0.016141519]
📊 Text 2 (length: 13 chars) → Embedding dim: 768
💬 First 5 values: [0.0041471613, -0.0067517012, -0.037806608, -0.031112626, -0.01571266]
📊 Text 3 (length: 5 chars) → Embedding dim: 768
💬 First 5 values: [-0.03290568, -0.0094595, 0.0029125062, 0.025114521, 0.0035863265]
📊 Text 4 (length: 200 chars) → Embedding dim: 768
💬 First 5 values: [-0.00843101, 0.018219126, -0.07413369, 0.06191114, 0.023585865]


In [11]:
# Check the data consistency
# text-embedding-004: 768 dimensions
if len(set(dimensions)) == 1:
    success(f"All embeddings have consistent dimension: {dimensions[0]}")
else:
    warn(f"Inconsistent dimensions found: {set(dimensions)}")

data(f"Embedding dimension: {dimensions[0]}")

✅ All embeddings have consistent dimension: 768
📊 Embedding dimension: 768


## ✅ Step 05 — API Quota Check

### 🤖 Gemini 2.5 Flash (Text Generation)

| Metric | Limit | Status |
|--------|-------|--------|
| **RPM** (Requests Per Minute) | 5 | ⚠️ Low |
| **TPM** (Tokens Per Minute) | 250K | ✅ OK |
| **RPD** (Requests Per Day) | 20 | ⚠️ Limited |
| **Use Case** | Generate answers | Development OK |

---

### 🧮 Text Embedding 004 (Embeddings)

| Metric | Limit | Status |
|--------|-------|--------|
| **RPM** | Unknown | ✅ Tested OK |
| **TPM** | Unknown | ✅ Tested OK |
| **RPD** | No strict limit | ✅ 25+ calls successful |
| **Dimension** | 768 | ✅ Consistent |
| **Use Case** | Convert documents to vectors | ✅ Sufficient |

---

### 📊 Project Requirements

| Item | Value |
|------|-------|
| **Total Documents** | 5,086 |
| **One-time Embedding Calls** | 5,086 |
| **Daily LLM Calls (Dev)** | ~10-20 |
| **Daily LLM Calls (Prod)** | Depends on users |

---

### 💡 Recommendations

✅ **Embedding API**: Safe to use for all 5,086 documents  
⚠️ **LLM API**: 20/day limit OK for development, may need upgrade for production  
💰 **Cost**: $0 (Free tier, no credit card required)

---

### 🎉 Summary

- ✅ All API tests passed
- ✅ Embedding API has sufficient quota
- ✅ LLM API works but has daily limits
- ✅ Ready to proceed with Chapter 04 (Vector Database Setup)